## 參考
- [BST - keras.io](https://keras.io/examples/structured_data/movielens_recommendations_transformers/)

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers

In [71]:
# 讀取資料 from .dat

data_dir = 'ml-1m/ml-1m'

users = pd.read_csv(data_dir+'/users.dat', delimiter='::', names=['user_id', 'gender', 'age', 'occupation', 'zip_code'])
movies = pd.read_csv(data_dir+'/movies.dat', delimiter='::', names=['movie_id', 'title', 'genres'])
ratings = pd.read_csv(data_dir+'/ratings.dat', delimiter='::', names=['user_id', 'movie_id', 'rating', 'timestamp'])

C:\Users\aband\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
C:\Users\aband\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\Users\aband\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


In [72]:
users.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [73]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [74]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [75]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1000209 non-null  int64
 1   movie_id   1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [76]:
# 先透過df frame 排序, 之後用group by就可以得到正確順序(會保持原順序)

ratings.sort_values(by=['user_id', 'timestamp'], inplace=True, ascending=True)

In [77]:
ratings

,user_id,movie_id,rating,timestamp
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


In [79]:
# 取得重要參數, 就是總movie 數目, +1即可用於[mask]

num_movies = len(movies['movie_id'].unique())
num_users = len(users['user_id'].unique())

print(f'電影數量: {num_movies}\n使用者數量: {num_users}')

電影數量: 3883
使用者數量: 6040


In [83]:
# 中間有斷, 有點麻煩, 需要重新排序, 或者將數量提升, 沒有的就沒用到而已, okay這樣比較方便。

num_movies = movies['movie_id'].max() + 1
mask_token_id = movies['movie_id'].max() + 1
movies['movie_id'].max()

3952

In [93]:
# 整理成論文需要的資料
# [movie1, movie2, movie3, movie4, movie5...100]
# N = 100
from pprint import pprint

# 透過ratings, 利用groupby user_id, timestamp


groups = ratings.groupby(by=['user_id']).groups

In [95]:
lengths = 0

for key in groups:
    print(f'user_id(${key}): 長度(${len(groups[key])})')
    lengths += len(groups[key])
    print('-'*15)
    pprint(groups[key])

user_id($1): 長度($53)
---------------
Int64Index([31, 22, 27, 37, 24, 36,  3,  7, 47,  0, 21, 44,  9, 51, 43, 41, 48,
            18, 11, 14, 42, 17, 39, 45, 26,  2,  6, 19, 38, 52,  1, 13, 49, 50,
            15, 20, 46,  5,  8, 12, 28, 23, 10, 16, 29, 33, 40,  4, 30, 35, 32,
            34, 25],
           dtype='int64')
user_id($2): 長度($129)
---------------
Int64Index([130,  64,  71, 134,  88, 170, 106, 120, 172,  70,
            ...
             92, 129, 167,  74,  80, 133,  66,  73,  87, 136],
           dtype='int64', length=129)
user_id($3): 長度($51)
---------------
Int64Index([217, 202, 186, 230, 190, 225, 209, 216, 218, 226, 212, 201, 211,
            228, 214, 215, 223, 189, 195, 198, 199, 205, 207, 194, 197, 210,
            227, 222, 229, 204, 206, 224, 184, 188, 231, 203, 182, 185, 193,
            220, 200, 221, 192, 196, 208, 183, 213, 219, 187, 191, 232],
           dtype='int64')
user_id($4): 長度($21)
---------------
Int64Index([234, 244, 233, 240, 249, 238, 241, 242, 248

---------------
Int64Index([30182, 30190, 30230, 30245, 30275, 30225, 30292, 30176, 30294,
            30266,
            ...
            30174, 30260, 30173, 30290, 30213, 30205, 30233, 30281, 30228,
            30244],
           dtype='int64', length=127)
user_id($204): 長度($446)
---------------
Int64Index([30610, 30649, 30679, 30715, 30396, 30709, 30304, 30428, 30510,
            30677,
            ...
            30723, 30514, 30606, 30353, 30385, 30483, 30555, 30685, 30517,
            30564],
           dtype='int64', length=446)
user_id($205): 長度($153)
---------------
Int64Index([30749, 30809, 30845, 30846, 30871, 30746, 30767, 30780, 30873,
            30884,
            ...
            30837, 30861, 30891, 30862, 30762, 30805, 30821, 30836, 30881,
            30801],
           dtype='int64', length=153)
user_id($206): 長度($30)
---------------
Int64Index([30900, 30904, 30917, 30899, 30902, 30907, 30910, 30912, 30914,
            30916, 30920, 30922, 30905, 30927, 30898, 30908, 

Int64Index([62083, 62097, 62127, 62079, 62099, 62107, 62138, 62070, 62089,
            62111,
            ...
            62166, 62122, 62133, 62168, 62125, 62126, 62135, 62145, 62090,
            62071],
           dtype='int64', length=108)
user_id($420): 長度($64)
---------------
Int64Index([62201, 62238, 62208, 62216, 62203, 62223, 62177, 62186, 62185,
            62190, 62219, 62228, 62229, 62195, 62232, 62182, 62183, 62213,
            62224, 62236, 62239, 62198, 62209, 62180, 62227, 62187, 62188,
            62206, 62231, 62233, 62221, 62217, 62220, 62235, 62197, 62200,
            62222, 62192, 62193, 62199, 62211, 62237, 62212, 62218, 62178,
            62184, 62204, 62207, 62214, 62234, 62240, 62205, 62181, 62191,
            62196, 62226, 62179, 62194, 62202, 62210, 62215, 62225, 62189,
            62230],
           dtype='int64')
user_id($421): 長度($20)
---------------
Int64Index([62245, 62248, 62255, 62257, 62259, 62260, 62256, 62251, 62253,
            62243, 62250, 62254, 

user_id($615): 長度($94)
---------------
Int64Index([92272, 92304, 92312, 92253, 92287, 92333, 92315, 92324, 92284,
            92273, 92281, 92292, 92285, 92243, 92262, 92275, 92276, 92286,
            92327, 92279, 92251, 92254, 92289, 92300, 92326, 92267, 92293,
            92332, 92258, 92331, 92335, 92269, 92261, 92319, 92249, 92325,
            92329, 92265, 92252, 92302, 92256, 92257, 92299, 92336, 92320,
            92244, 92296, 92322, 92271, 92314, 92259, 92255, 92282, 92277,
            92283, 92309, 92250, 92245, 92303, 92317, 92248, 92298, 92328,
            92308, 92247, 92263, 92311, 92321, 92260, 92306, 92278, 92323,
            92274, 92246, 92297, 92290, 92310, 92313, 92316, 92266, 92295,
            92307, 92291, 92318, 92280, 92288, 92294, 92305, 92264, 92270,
            92330, 92268, 92334, 92301],
           dtype='int64')
user_id($616): 長度($44)
---------------
Int64Index([92349, 92359, 92367, 92379, 92370, 92339, 92347, 92353, 92374,
            92338, 92341, 9234

           dtype='int64', length=126)
user_id($844): 長度($84)
---------------
Int64Index([130295, 130311, 130333, 130334, 130306, 130272, 130348, 130289,
            130296, 130305, 130273, 130297, 130325, 130307, 130299, 130345,
            130326, 130281, 130294, 130284, 130274, 130319, 130280, 130353,
            130283, 130339, 130288, 130271, 130276, 130300, 130338, 130344,
            130318, 130298, 130291, 130308, 130323, 130270, 130315, 130340,
            130352, 130349, 130310, 130350, 130316, 130330, 130287, 130278,
            130279, 130282, 130347, 130335, 130332, 130301, 130312, 130329,
            130324, 130336, 130286, 130314, 130337, 130341, 130346, 130351,
            130302, 130309, 130317, 130285, 130293, 130303, 130313, 130327,
            130342, 130277, 130275, 130292, 130290, 130321, 130343, 130322,
            130331, 130328, 130304, 130320],
           dtype='int64')
user_id($845): 長度($52)
---------------
Int64Index([130355, 130363, 130371, 130365, 130374, 1

...

---------------
Int64Index([840396, 840424, 840434, 840466, 840404, 840460, 840511, 840467,
            840401, 840512,
            ...
            840515, 840431, 840530, 840550, 840561, 840517, 840537, 840486,
            840443, 840463],
           dtype='int64', length=192)
user_id($5050): 長度($85)
---------------
Int64Index([840626, 840575, 840647, 840650, 840581, 840605, 840618, 840629,
            840589, 840599, 840612, 840648, 840583, 840573, 840654, 840637,
            840639, 840632, 840609, 840627, 840651, 840625, 840642, 840587,
            840602, 840606, 840634, 840577, 840604, 840614, 840620, 840631,
            840580, 840610, 840613, 840635, 840571, 840588, 840596, 840607,
            840619, 840630, 840646, 840576, 840623, 840628, 840572, 840579,
            840584, 840598, 840617, 840633, 840582, 840603, 840643, 840591,
            840638, 840585, 840641, 840574, 840601, 840586, 840608, 840611,
            840616, 840644, 840593, 840594, 840645, 840649, 840615, 84059

           dtype='int64')
user_id($5300): 長度($282)
---------------
Int64Index([877068, 877045, 876950, 876965, 877024, 876925, 876926, 876969,
            877015, 877091,
            ...
            877196, 876964, 876971, 877199, 876932, 876933, 876968, 876983,
            877066, 877075],
           dtype='int64', length=282)
user_id($5301): 長度($139)
---------------
Int64Index([877222, 877262, 877224, 877240, 877220, 877221, 877232, 877237,
            877334, 877294,
            ...
            877317, 877202, 877266, 877208, 877287, 877278, 877274, 877212,
            877286, 877282],
           dtype='int64', length=139)
user_id($5302): 長度($238)
---------------
Int64Index([877361, 877558, 877475, 877472, 877391, 877415, 877345, 877348,
            877356, 877469,
            ...
            877539, 877540, 877548, 877394, 877534, 877544, 877354, 877478,
            877427, 877511],
           dtype='int64', length=238)
user_id($5303): 長度($45)
---------------
Int64Index([877615, 87

Int64Index([914415, 914457, 914459, 914389, 914409, 914410, 914412, 914423,
            914388, 914405, 914426, 914452, 914454, 914394, 914397, 914399,
            914428, 914395, 914411, 914432, 914444, 914387, 914396, 914419,
            914443, 914447, 914391, 914453, 914390, 914430, 914455, 914398,
            914404, 914417, 914424, 914445, 914393, 914436, 914460, 914413,
            914400, 914420, 914427, 914434, 914442, 914449, 914418, 914431,
            914438, 914416, 914446, 914456, 914402, 914425, 914441, 914451,
            914458, 914429, 914392, 914401, 914422, 914439, 914408, 914435,
            914437, 914407, 914433, 914450, 914461, 914406, 914414, 914421,
            914448, 914403, 914440],
           dtype='int64')
user_id($5529): 長度($21)
---------------
Int64Index([914462, 914463, 914464, 914465, 914468, 914473, 914476, 914480,
            914481, 914482, 914466, 914467, 914469, 914470, 914471, 914474,
            914475, 914477, 914478, 914479, 914472],
        

           dtype='int64')
user_id($5747): 長度($564)
---------------
Int64Index([950938, 951052, 951080, 951200, 951306, 951141, 951142, 951301,
            951303, 951304,
            ...
            951447, 951297, 951456, 951072, 951057, 951243, 951240, 951448,
            950996, 951441],
           dtype='int64', length=564)
user_id($5748): 長度($54)
---------------
Int64Index([951495, 951519, 951503, 951510, 951529, 951532, 951500, 951523,
            951524, 951488, 951504, 951499, 951494, 951511, 951526, 951533,
            951506, 951483, 951491, 951520, 951490, 951482, 951509, 951525,
            951518, 951534, 951486, 951501, 951530, 951502, 951508, 951498,
            951513, 951514, 951489, 951527, 951528, 951484, 951481, 951507,
            951512, 951497, 951493, 951505, 951515, 951487, 951516, 951485,
            951522, 951517, 951492, 951531, 951521, 951496],
           dtype='int64')
user_id($5749): 長度($407)
---------------
Int64Index([951569, 951667, 951677, 951700, 95

           dtype='int64', length=142)
user_id($5948): 長度($398)
---------------
Int64Index([984145, 984221, 983965, 984180, 984274, 983963, 984115, 984162,
            983921, 983931,
            ...
            983996, 984079, 984043, 984001, 984067, 984182, 984003, 984268,
            984025, 983967],
           dtype='int64', length=398)
user_id($5949): 長度($181)
---------------
Int64Index([984391, 984364, 984388, 984437, 984365, 984297, 984412, 984381,
            984459, 984405,
            ...
            984382, 984409, 984372, 984474, 984368, 984362, 984329, 984385,
            984325, 984460],
           dtype='int64', length=181)
user_id($5950): 長度($435)
---------------
Int64Index([984822, 984909, 984761, 984588, 984717, 984853, 984876, 984534,
            984857, 984700,
            ...
            984847, 984842, 984651, 984598, 984597, 984731, 984682, 984475,
            984660, 984733],
           dtype='int64', length=435)
user_id($5951): 長度($53)
---------------
Int64Index

In [96]:
# 抓出單一個group

list(groups[1])

[31,
 22,
 27,
 37,
 24,
 36,
 3,
 7,
 47,
 0,
 21,
 44,
 9,
 51,
 43,
 41,
 48,
 18,
 11,
 14,
 42,
 17,
 39,
 45,
 26,
 2,
 6,
 19,
 38,
 52,
 1,
 13,
 49,
 50,
 15,
 20,
 46,
 5,
 8,
 12,
 28,
 23,
 10,
 16,
 29,
 33,
 40,
 4,
 30,
 35,
 32,
 34,
 25]

In [97]:
lengths

1000209

In [98]:
# 平均看影片長度

lengths // len(groups)

165

In [99]:
# 大概會有幾筆資料

mean_length = 120
L = 50
num_users = len(users)

(mean_length - L) * num_users * 0.15

63420.0

In [ ]:
# 輸入長度L: 50
# 不足長度怎麼處理?
# 1. 補0
# 2. 不使用
# 爆了, 檔案太大!


L = 50

def get_raw_input(groups):
    global L
    rows, temp = [], []
    for key in groups:
        if len(groups[key]) < L:
            continue
            # temp = list(groups) + [0]*(L-len(groups[key]))
        else:
            temp = list(groups)
        for i in range(0, len(temp)-L):
            # 將資料包裝
            rows.append(temp[i:i+L])
            
    return rows


rows = get_raw_input(groups)

In [ ]:
for i in range(3):
    pprint(rows[i])

## 模型架構
- Encoder part
- Classifier part

In [84]:
# config, 透過dataclass decorator自動加入__init__

from dataclasses import dataclass


@dataclass
class BERT4RecConfig:
    max_len: int=128
    batch_size: int=128
    lr: float=0.001
    item_size: int=num_movies
    embed_dim: int=16
    num_heads: int=8
    ff_dim: int=16
    num_layers: int=2
    dropout_rate: float=0.2
    
    
config = BERT4RecConfig()
config

BERT4RecConfig(max_len=128, batch_size=128, lr=0.001, item_size=3953, embed_dim=16, num_heads=8, ff_dim=16, num_layers=2, dropout_rate=0.2)

![Trm - 
Encoder](trm_encoder.png)

---

![image.png](bert4rec.png)




In [85]:
def trm_encoder(q, k, v, i):
    """
        Encoder part
    """
    attention_output = layers.MultiHeadAttention(
        num_heads=config.num_heads,
        key_dim=config.embed_dim // config.num_heads,
        name=f'encoder_{i}/multiheadattention'
    )(q, k, v)
    attention_output = layers.Dropout(rate=config.dropout_rate, name=f'encoder_{i}/att_dropout')(attention_output)
    attention_output = layers.LayerNormalization(
        epsilon=1e-1, name=f'encoder_{i}/att_layernormalization'
    )(q + attention_output)                 # short-cut

    # ffn
    ffn = keras.Sequential([
        layers.Dense(config.ff_dim, activation='relu'),
        layers.Dense(config.embed_dim),          
    ], name=f"encoder_{i}/ffn")
    ffn_output = ffn(attention_output)
    ffn_output = layers.Dropout(0.1, name=f"encoder_{i}/ffn_dropout")(
        ffn_output
    )
    sequence_output = layers.LayerNormalization(
        epsilon=1e-6, name=f"encoder_{i}/ffn_layernormalization"
    )(attention_output + ffn_output)            # short-cut
    return sequence_output

In [86]:
# 透過繼承於 keras.Model 去細緻調整fit過程
loss_fn = keras.losses.SparseCategoricalCrossentropy(
    reduction=tf.keras.losses.Reduction.NONE
)
loss_tracker = tf.keras.metrics.Mean(name="loss")


class BERT4RecMLM(keras.Model):
    def train_step(self, inputs):
        if len(inputs) == 2:
            features, labels = inputs
            sample_weight = None
        elif len(inputs) == 3:
            features, labels, sample_weight = inputs
        else:
            print('輸入有誤...(BERT4RecMLM)')
        
        with tf.GradientTape() as tape:
            predictions = self(features, training=True)
            loss = loss_fn(labels, predictions, sample_weight=sample_weight)
        
        # 計算梯度
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        
        # 參數更新
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        # 計算 metrics
        loss_tracker.update_state(loss, sample_weight=sample_weight)
        
        return {'loss': loss_tracker.result()}
    
    @property
    def metrics(self):
        return [loss_tracker]

In [87]:
def create_bert4rec():
    # input
    inputs = keras.Input(shape=(config.max_len,), dtype='int64', name='input')
    # Embedding(item + pos)
    item_embedding_layer = layers.Embedding(
        input_dim=config.item_size,
        output_dim=config.embed_dim,
        name='item_embedding'
    )
    item_embedding = item_embedding_layer(inputs)
    pos_embedding = layers.Embedding(
        input_dim=config.max_len,
        output_dim=config.embed_dim,
        name='pos_embedding'
    )(tf.range(start=0, limit=config.max_len, delta=1))
    embedding = item_embedding + pos_embedding
    
    # Encoder
    for i in range(config.num_layers):
        embedding = trm_encoder(embedding, embedding, embedding, i)
    
    # output layer, share embedding(這邊不是layer應該不能直接操作...不符合functional API) ---> 發現可以!!!
    outputs = tf.matmul(embedding, tf.transpose(a=item_embedding_layer.trainable_variables[0]))
    outputs = layers.Activation('softmax')(outputs)
    
    model = BERT4RecMLM(inputs, outputs)
    model.compile(optimizer=keras.optimizers.Adam(config.lr))
    return model

    
bert4rec = create_bert4rec()
bert4rec.summary()

Model: "ber_t4rec_mlm_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 128)]        0                                            
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 128, 16)      63248       input[0][0]                      
__________________________________________________________________________________________________
tf.__operators__.add_26 (TFOpLa (None, 128, 16)      0           item_embedding[0][0]             
__________________________________________________________________________________________________
encoder_0/multiheadattention (M (None, 128, 16)      1088        tf.__operators__.add_26[0][0]    
                                                                 tf.__operators__.ad

## embedding 輸入輸出共享測試

In [88]:
# 找出 共享 embedding 方法, 需要轉置
emd = layers.Embedding(10000, 32)
embedding = emd(np.array([1,2, 3]))

emd.weights

[<tf.Variable 'embedding_9/embeddings:0' shape=(10000, 32) dtype=float32, numpy=
 array([[-0.04565385,  0.0228415 ,  0.01408012, ...,  0.02277399,
          0.03881348, -0.02892498],
        [ 0.00531975,  0.01409591,  0.04157652, ..., -0.02709459,
         -0.04011021, -0.02030834],
        [ 0.04395641,  0.03908107,  0.03616558, ..., -0.04538272,
         -0.02270526, -0.04050515],
        ...,
        [ 0.03277861,  0.04361615,  0.03682769, ..., -0.0142963 ,
         -0.00953268,  0.00386209],
        [-0.04249771,  0.02699311, -0.03973543, ..., -0.04423676,
          0.01094314,  0.03270457],
        [ 0.00862191,  0.01632955,  0.03811376, ...,  0.04757805,
          0.00394714,  0.04658416]], dtype=float32)>]

In [89]:
# 轉置! 但這需要該layer 要記住名稱

tf.transpose(a=emd.trainable_variables[0])

<tf.Tensor: shape=(32, 10000), dtype=float32, numpy=
array([[-0.04565385,  0.00531975,  0.04395641, ...,  0.03277861,
        -0.04249771,  0.00862191],
       [ 0.0228415 ,  0.01409591,  0.03908107, ...,  0.04361615,
         0.02699311,  0.01632955],
       [ 0.01408012,  0.04157652,  0.03616558, ...,  0.03682769,
        -0.03973543,  0.03811376],
       ...,
       [ 0.02277399, -0.02709459, -0.04538272, ..., -0.0142963 ,
        -0.04423676,  0.04757805],
       [ 0.03881348, -0.04011021, -0.02270526, ..., -0.00953268,
         0.01094314,  0.00394714],
       [-0.02892498, -0.02030834, -0.04050515, ...,  0.00386209,
         0.03270457,  0.04658416]], dtype=float32)>

In [90]:
# 可以使用!, 但應該要在call裡面使用, 所以需要用繼承keras.Model模式來寫!

tf.matmul(embedding, tf.transpose(a=emd.trainable_variables[0]))

<tf.Tensor: shape=(3, 10000), dtype=float32, numpy=
array([[ 0.00295702,  0.02385878,  0.00536622, ...,  0.00704855,
         0.00206965,  0.00226169],
       [-0.00061949,  0.00536622,  0.02979579, ..., -0.00245284,
        -0.00624401,  0.00031275],
       [ 0.00046163, -0.00302564, -0.0011524 , ..., -0.00433287,
         0.00389158,  0.00151961]], dtype=float32)>

In [91]:
# model 輸入測試確保可以使用

inputs = np.array([[1000]*128], dtype=np.int64)
bert4rec(inputs)

<tf.Tensor: shape=(1, 128, 3953), dtype=float32, numpy=
array([[[0.00020181, 0.00022673, 0.00022804, ..., 0.00030195,
         0.00026357, 0.00021186],
        [0.00021734, 0.00025212, 0.00020987, ..., 0.00028276,
         0.00026578, 0.00024568],
        [0.0002291 , 0.00020995, 0.00027308, ..., 0.00027523,
         0.0002666 , 0.00022405],
        ...,
        [0.00022932, 0.00021925, 0.00025138, ..., 0.0002734 ,
         0.00029029, 0.00022199],
        [0.00022149, 0.0002334 , 0.00020401, ..., 0.00027082,
         0.00031555, 0.00025754],
        [0.00022589, 0.00023743, 0.00023314, ..., 0.00027141,
         0.00026577, 0.00022947]]], dtype=float32)>

In [92]:
# 預測

np.argmax(tf.nn.softmax(bert4rec(inputs)), axis=2)

array([[3573, 3768, 3682,   73, 3719, 2715, 3573,  122,  377, 3573, 3573,
        3573, 3299, 3573, 2356, 3573, 3573, 3573, 2204, 1109,   73, 1304,
        3573, 1109,  449, 3573, 2343, 1304, 3118, 1073, 3573, 2343, 3573,
          73, 1861,  903, 1007,  733, 3385, 3573, 1109,  810, 3573, 1109,
        2343, 1290, 3901, 1304, 1109, 3385, 3573,  377, 3573, 1304, 1382,
        3719, 2285,   73, 3573, 3573, 1085, 3441, 1011, 1992,  794, 2715,
        2385,  794, 3573,  733,  733,   73, 1534, 3573, 3026, 2382, 3131,
        1109,  794, 2715, 3573, 2715, 3118, 2193, 3573, 2718,  377, 3078,
        3118,  377, 3573, 1304, 3056,  903, 2715, 1109, 1304, 1109, 3385,
        3573,   73, 2343, 1109, 3573, 1853, 3719, 3719, 3594, 2343, 3901,
        1109, 2715, 3573, 2382, 2382, 3573,   73,  429, 3573,   73, 3901,
        2969, 1304, 3573, 1109, 1109, 3476,  903]], dtype=int64)

## **接著需要**

- 將[mask] token 放入 vocabulary(可以採用與text 同樣方式, 透過TextVectorization方式 or not)
- 將輸入資料mask: portion = 0.4
- 測試訓練
- 負樣本
- 正式訓練

In [1]:
# [mask] token ---> ok

# 將輸入資料mask: 先將資料轉換成seq data, 但要直接用numpy 還是 用成 檔案閱讀 成dataset呢?
# 以擴充來說, dataset更好, 但目前以實現為原則, 先用numpy ---> 記憶體爆開
# 那採取 寫成txt檔案, user_id,裡面每一row為 [movied_id1, movie_id2,  ... movied_id_length]




NameError: name 'config' is not defined